# DenseNet201-SE Standalone (Paddy Disease Classification)

Notebook ini adalah implementasi **DenseNet201-SE (Squeeze & Excitation)** murni yang dioptimalkan untuk:
1.  **Keras 3 Compatible:** Menggunakan `@register_keras_serializable` untuk layer custom (aman disimpan/diload).
2.  **Dynamic Dataset Loading:** Membaca langsung dari folder `train_images`, memastikan **14.112 data augmentasi baru** terbaca sempurna.
3.  **Balanced Training:** Memanfaatkan dataset yang sudah seimbang (1.764 gambar/kelas).
4.  **2-Stage Training:** Transfer Learning (Freeze Backbone) -> Fine Tuning (Unfreeze Last Layers).

**Arsitektur:** DenseNet201 + SE Block + GlobalAveragePooling + Dropout + Dense Softmax.

In [ ]:
import os
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import tensorflow as tf
import keras
from keras import layers

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)

In [ ]:
# ----------------------------
# 1) Configuration
# ----------------------------
DATASET_DIR   = "paddy-disease-classification"
TRAIN_IMG_DIR = os.path.join(DATASET_DIR, "train_images")
TEST_IMG_DIR  = os.path.join(DATASET_DIR, "test_images")
SAMPLE_SUB    = os.path.join(DATASET_DIR, "sample_submission.csv")

OUTPUT_DIR = "outputs_densenet201_se_standalone"
os.makedirs(OUTPUT_DIR, exist_ok=True)

SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

IMG_SIZE   = (224, 224)
BATCH_SIZE = 16
AUTOTUNE   = tf.data.AUTOTUNE

EPOCHS_STAGE1 = 10
EPOCHS_STAGE2 = 15

LR1           = 1e-3
LR2           = 1e-5
UNFREEZE_LAST = 30

DROPOUT  = 0.4
SE_RATIO = 16

USE_FOCAL_LOSS = True
GAMMA = 2.0
ALPHA = 0.25

In [ ]:
# ----------------------------
# 2) Dynamic Dataset Loading (Folder Scan)
# ----------------------------
filepaths = []
labels    = []

classes = sorted(os.listdir(TRAIN_IMG_DIR))
classes = [c for c in classes if os.path.isdir(os.path.join(TRAIN_IMG_DIR, c))]
print(f"[INFO] Classes found ({len(classes)}): {classes}")

class_counts = {}
for label in classes:
    class_dir = os.path.join(TRAIN_IMG_DIR, label)
    images = [f for f in os.listdir(class_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    class_counts[label] = len(images)
    for img in images:
        filepaths.append(os.path.join(class_dir, img))
        labels.append(label)

df = pd.DataFrame({'filepath': filepaths, 'label': labels})

print(f"\n[INFO] Total Dataset: {len(df)} images")
print("Distribution per class:")
print(pd.Series(class_counts))

class_names  = sorted(df['label'].unique().tolist())
num_classes  = len(class_names)
class_to_idx = {c: i for i, c in enumerate(class_names)}

with open(os.path.join(OUTPUT_DIR, "class_names.json"), "w") as f:
    json.dump(class_names, f, indent=2)

In [ ]:
# ----------------------------
# 3) Train/Val Split (Stratified 90:10)
# ----------------------------
train_df, val_df = train_test_split(
    df, test_size=0.10, random_state=SEED, stratify=df["label"]
)
train_df = train_df.reset_index(drop=True)
val_df   = val_df.reset_index(drop=True)

print(f"Training Set   : {len(train_df)} images")
print(f"Validation Set : {len(val_df)} images")

In [ ]:
# ----------------------------
# 4) tf.data Input Pipeline
# ----------------------------
def decode_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE, method="bilinear")
    return tf.cast(img, tf.float32)

def process_path(path, label_idx):
    return decode_image(path), tf.one_hot(label_idx, num_classes)

def make_ds(dataframe, training=True):
    paths         = dataframe["filepath"].values
    label_indices = dataframe["label"].map(class_to_idx).values.astype('int32')
    ds = tf.data.Dataset.from_tensor_slices((paths, label_indices))
    if training:
        ds = ds.shuffle(buffer_size=min(len(dataframe), 5000), seed=SEED)
    ds = ds.map(process_path, num_parallel_calls=AUTOTUNE)
    return ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

train_ds = make_ds(train_df, training=True)
val_ds   = make_ds(val_df, training=False)

In [ ]:
# ----------------------------
# 5) Keras 3 Serializable Components
# ----------------------------
@keras.saving.register_keras_serializable(package="custom")
class FocalLoss(tf.keras.losses.Loss):
    def __init__(self, gamma=2.0, alpha=0.25, from_logits=False, name="focal_loss"):
        super().__init__(name=name)
        self.gamma = gamma
        self.alpha = alpha
        self.from_logits = from_logits

    def call(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        if self.from_logits:
            y_pred = tf.nn.softmax(y_pred)
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
        ce     = -y_true * tf.math.log(y_pred)
        weight = self.alpha * tf.pow(1.0 - y_pred, self.gamma)
        return tf.reduce_sum(weight * ce, axis=-1)

    def get_config(self):
        return {"gamma": self.gamma, "alpha": self.alpha, "from_logits": self.from_logits, "name": self.name}

@keras.saving.register_keras_serializable(package="custom")
class DenseNetPreprocess(tf.keras.layers.Layer):
    def call(self, x):
        return tf.keras.applications.densenet.preprocess_input(x)
    def get_config(self):
        return {}

LOSS_FN = FocalLoss(gamma=GAMMA, alpha=ALPHA) if USE_FOCAL_LOSS else "categorical_crossentropy"
print("Loss Function:", LOSS_FN.name if hasattr(LOSS_FN, 'name') else LOSS_FN)

In [ ]:
# ----------------------------
# 6) Model Architecture (DenseNet201 + SE Block)
# ----------------------------
def se_block(x, ratio=16, name="se"):
    c  = int(x.shape[-1])
    se = tf.keras.layers.GlobalAveragePooling2D(name=f"{name}_gap")(x)
    se = tf.keras.layers.Dense(max(1, c // ratio), activation="relu",    name=f"{name}_fc1")(se)
    se = tf.keras.layers.Dense(c,                  activation="sigmoid", name=f"{name}_fc2")(se)
    se = tf.keras.layers.Reshape((1, 1, c),                              name=f"{name}_reshape")(se)
    return tf.keras.layers.Multiply(name=f"{name}_scale")([x, se])

def build_densenet201_se(num_classes, dropout=DROPOUT, se_ratio=SE_RATIO, name="DenseNet201_SE"):
    backbone = tf.keras.applications.DenseNet201(
        include_top=False, weights="imagenet",
        input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
    )
    backbone.trainable = False

    inp = tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
    x   = DenseNetPreprocess(name="densenet_preprocess")(inp)
    x   = backbone(x, training=False)
    x   = se_block(x, ratio=se_ratio, name="se_block")
    x   = tf.keras.layers.GlobalAveragePooling2D()(x)
    x   = tf.keras.layers.Dropout(dropout)(x)
    out = tf.keras.layers.Dense(num_classes, activation="softmax")(x)
    return tf.keras.Model(inp, out, name=name)

model = build_densenet201_se(num_classes)
model.compile(optimizer=tf.keras.optimizers.Adam(LR1), loss=LOSS_FN, metrics=["accuracy"])
model.summary()

In [ ]:
# ----------------------------
# 7) Class Weights
# ----------------------------
classes_idx  = train_df["label"].map(class_to_idx).values
cw           = compute_class_weight(class_weight="balanced", classes=np.unique(classes_idx), y=classes_idx)
class_weight = {i: float(w) for i, w in enumerate(cw)}
print("Class Weights:", class_weight)

In [ ]:
# ----------------------------
# 8) Train & Eval Plot Helper
# ----------------------------
def plot_history(history, stage_name="Training"):
    acc      = history.history['accuracy']
    val_acc  = history.history['val_accuracy']
    loss     = history.history['loss']
    val_loss = history.history['val_loss']
    epochs   = range(1, len(acc) + 1)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))
    fig.suptitle(f"Train Evaluation — {stage_name}", fontsize=14, fontweight='bold')

    ax1.plot(epochs, acc,     'bo-', label='Train Accuracy')
    ax1.plot(epochs, val_acc, 'ro-', label='Val Accuracy')
    ax1.set_title('Accuracy'); ax1.set_xlabel('Epoch'); ax1.set_ylabel('Accuracy')
    ax1.legend(); ax1.grid(True, alpha=0.3)

    ax2.plot(epochs, loss,     'bo-', label='Train Loss')
    ax2.plot(epochs, val_loss, 'ro-', label='Val Loss')
    ax2.set_title('Loss'); ax2.set_xlabel('Epoch'); ax2.set_ylabel('Loss')
    ax2.legend(); ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

In [ ]:
# ----------------------------
# 9) STAGE 1: Train Head Only
# ----------------------------
checkpoint_path = os.path.join(OUTPUT_DIR, "best_stage1.keras")

callbacks_stage1 = [
    tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor="val_accuracy", save_best_only=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1)
]

print("\n🚀 Starting Stage 1 Training (Head Only)...")
history1 = model.fit(
    train_ds, validation_data=val_ds,
    epochs=EPOCHS_STAGE1, callbacks=callbacks_stage1, class_weight=class_weight
)
plot_history(history1, "Stage 1 — Head Training")

In [ ]:
# ----------------------------
# 10) STAGE 2: Fine Tuning
# ----------------------------
model = tf.keras.models.load_model(
    checkpoint_path,
    custom_objects={"FocalLoss": FocalLoss, "DenseNetPreprocess": DenseNetPreprocess}
)

backbone = model.layers[2]
backbone.trainable = True

for layer in backbone.layers[:-UNFREEZE_LAST]:
    layer.trainable = False
for layer in backbone.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(LR2), loss=LOSS_FN, metrics=["accuracy"])

final_checkpoint_path = os.path.join(OUTPUT_DIR, "densenet201_se_final.keras")

callbacks_stage2 = [
    tf.keras.callbacks.ModelCheckpoint(final_checkpoint_path, monitor="val_accuracy", save_best_only=True, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=4, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1)
]

print(f"\n🚀 Starting Stage 2 Fine-Tuning (Unfreezing last {UNFREEZE_LAST} layers)...")
history2 = model.fit(
    train_ds, validation_data=val_ds,
    epochs=EPOCHS_STAGE2, callbacks=callbacks_stage2, class_weight=class_weight
)
plot_history(history2, "Stage 2 — Fine Tuning")

In [ ]:
# ----------------------------
# 11) Evaluasi: Confusion Matrix & Report
# ----------------------------
best_model = tf.keras.models.load_model(
    final_checkpoint_path,
    custom_objects={"FocalLoss": FocalLoss, "DenseNetPreprocess": DenseNetPreprocess}
)

print("\n📊 Generating Evaluation Metrics...")

y_pred, y_true = [], []
for bx, by in val_ds:
    probs = best_model.predict(bx, verbose=0)
    y_pred.extend(np.argmax(probs, axis=1))
    y_true.extend(np.argmax(by.numpy(), axis=1))

print("\n✅ Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(11, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted'); plt.ylabel('Ground Truth')
plt.title('Confusion Matrix — Validation Set')
plt.tight_layout()
plt.show()

In [ ]:
# ----------------------------
# 12) Save Model
# ----------------------------
print("\n💾 Model saved successfully at:")
print(f"   - {final_checkpoint_path}")
print("   (Gunakan file ini untuk inference atau deployment)")

---
## 🔍 13) Inference Helper

Bagian ini digunakan untuk **menguji model** pada gambar baru setelah proses training selesai.
**Tidak perlu training ulang** — cukup load model yang sudah tersimpan dan jalankan prediksi.

### Cara menjalankan:

| Urutan | Cell | Fungsi |
|:---:|---|---|
| **1** | **Cell 13a** | Load semua library + model + class names |
| **2** | **Cell 13b** | Prediksi satu gambar dari folder `test_images/` (di luar dataset) |
| **3** | **Cell 13c** | Prediksi semua gambar di folder `test_images/` → simpan hasil ke CSV |

> ⚠️ **Cell 13a harus dijalankan pertama.** Letakkan gambar uji di folder `test_images/` (bukan di dalam `paddy-disease-classification/`).

In [ ]:
# ----------------------------
# 13a) Load Model + Library untuk Inference
# WAJIB dijalankan pertama sebelum Cell 13b / 13c
# ----------------------------
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import tensorflow as tf
import keras

# ============================================================
# CONFIG — Sesuaikan path di sini jika perlu
# ============================================================
MODEL_PATH       = "model/outputs_densenet201_se_standalone/densenet201_se_final.keras"
CLASS_NAMES_PATH = "model/outputs_densenet201_se_standalone/class_names.json"

# Folder gambar uji (di LUAR folder paddy-disease-classification)
CUSTOM_TEST_DIR  = "test_images"

IMG_SIZE = (224, 224)
# ============================================================

# Re-register custom components (wajib agar load_model berhasil)
@keras.saving.register_keras_serializable(package="custom")
class FocalLoss(tf.keras.losses.Loss):
    def __init__(self, gamma=2.0, alpha=0.25, from_logits=False, name="focal_loss"):
        super().__init__(name=name)
        self.gamma = gamma; self.alpha = alpha; self.from_logits = from_logits
    def call(self, y_true, y_pred):
        if self.from_logits: y_pred = tf.nn.softmax(y_pred)
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
        ce = -tf.cast(y_true, tf.float32) * tf.math.log(y_pred)
        return tf.reduce_sum(self.alpha * tf.pow(1.0 - y_pred, self.gamma) * ce, axis=-1)
    def get_config(self):
        return {"gamma": self.gamma, "alpha": self.alpha, "from_logits": self.from_logits, "name": self.name}

@keras.saving.register_keras_serializable(package="custom")
class DenseNetPreprocess(tf.keras.layers.Layer):
    def call(self, x): return tf.keras.applications.densenet.preprocess_input(x)
    def get_config(self): return {}

CUSTOM_OBJ = {"FocalLoss": FocalLoss, "DenseNetPreprocess": DenseNetPreprocess}

# Load model
print(f"[INFO] Loading model dari: {MODEL_PATH}")
inf_model = tf.keras.models.load_model(MODEL_PATH, custom_objects=CUSTOM_OBJ)
print("[INFO] ✅ Model berhasil di-load!")

# Load class names
with open(CLASS_NAMES_PATH, "r") as f:
    class_names_inf = json.load(f)

print(f"[INFO] Classes ({len(class_names_inf)}): {class_names_inf}")
print(f"[INFO] Folder gambar uji : '{CUSTOM_TEST_DIR}'")
print("\n✅ Siap untuk inference. Jalankan Cell 13b atau 13c.")

### 13b. Prediksi Satu Gambar

Letakkan gambar daun padi Anda di folder **`test_images/`** (sejajar dengan notebook ini, di luar folder dataset).

Lalu ubah nama file di variabel `IMAGE_FILE` sesuai nama gambar Anda.

**Contoh struktur folder:**
```
Model DenseNet-201/
├── test_images/          ← Taruh gambar di sini
│   ├── image.png
│   └── foto_daun.jpg
├── paddy-disease-classification/   ← Dataset Kaggle (jangan diubah)
└── densenet201_se_standalone_FIX_keras3.ipynb
```

In [ ]:
# ----------------------------
# 13b) Prediksi SATU Gambar
# ----------------------------
def predict_single_image(model, class_names, image_path, img_size=(224, 224)):
    import matplotlib.patches as mpatches

    if not os.path.exists(image_path):
        raise FileNotFoundError(
            f"❌ Gambar tidak ditemukan: '{image_path}'\n"
            f"   Pastikan file ada di folder '{CUSTOM_TEST_DIR}/'\n"
            f"   Contoh: taruh gambar di '{CUSTOM_TEST_DIR}/image.png' lalu set IMAGE_FILE = 'image.png'"
        )

    # Preprocessing
    img_raw     = tf.io.read_file(image_path)
    img         = tf.image.decode_image(img_raw, channels=3, expand_animations=False)
    img_display = img.numpy().astype('uint8')
    img         = tf.image.resize(img, img_size, method="bilinear")
    img         = tf.cast(img, tf.float32)
    img         = tf.expand_dims(img, axis=0)

    # Inference
    probs      = model.predict(img, verbose=0)[0]
    pred_idx   = int(np.argmax(probs))
    pred_label = class_names[pred_idx]
    confidence = float(probs[pred_idx])

    # Visualisasi
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle("Hasil Prediksi DenseNet201-SE", fontsize=14, fontweight='bold')

    ax1.imshow(img_display)
    ax1.axis('off')
    color = '#27ae60' if confidence >= 0.80 else ('#e67e22' if confidence >= 0.50 else '#e74c3c')
    ax1.set_title(
        f"🏷️  {pred_label.replace('_', ' ').title()}\nConfidence: {confidence*100:.2f}%",
        fontsize=13, fontweight='bold', color=color
    )

    colors = ['#27ae60' if i == pred_idx else '#3498db' for i in range(len(class_names))]
    bars   = ax2.barh(class_names, probs * 100, color=colors, edgecolor='white', height=0.6)
    ax2.set_xlabel('Probability (%)', fontsize=11)
    ax2.set_title('Distribusi Probabilitas Semua Kelas', fontsize=11)
    ax2.set_xlim(0, 105)
    for bar, val in zip(bars, probs):
        ax2.text(val * 100 + 0.8, bar.get_y() + bar.get_height() / 2,
                 f'{val*100:.1f}%', va='center', fontsize=9)

    green_patch = mpatches.Patch(color='#27ae60', label='Predicted Class')
    blue_patch  = mpatches.Patch(color='#3498db', label='Other Classes')
    ax2.legend(handles=[green_patch, blue_patch], loc='lower right')
    ax2.grid(axis='x', alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"\n🎯 Hasil Prediksi:")
    print(f"   Kelas      : {pred_label}")
    print(f"   Confidence : {confidence*100:.2f}%")
    return pred_label, confidence, probs


# ===================================================
# GANTI nama file gambar di sini
# Gambar harus ada di folder 'test_images/'
IMAGE_FILE = "image.png"   # ← ganti sesuai nama file Anda
# ===================================================

IMAGE_PATH = os.path.join(CUSTOM_TEST_DIR, IMAGE_FILE)
print(f"[INFO] Memproses gambar: {IMAGE_PATH}")

pred_label, confidence, all_probs = predict_single_image(inf_model, class_names_inf, IMAGE_PATH)

### 13c. Prediksi Semua Gambar di `test_images/` → CSV

Cell ini akan memproses **semua file gambar** (`.jpg`, `.jpeg`, `.png`) yang ada di folder `test_images/` secara batch, lalu menyimpan hasilnya ke file CSV.

**Format output CSV:**
```
filename,predicted_label,confidence
image.png,blast,0.9732
foto_daun.jpg,normal,0.8811
```
File disimpan di: `test_images/prediction_results.csv`

In [ ]:
# ----------------------------
# 13c) Prediksi Semua Gambar di test_images/ → CSV
# ----------------------------
def predict_all_in_folder(model, class_names, test_dir, img_size=(224, 224), batch_size=16):
    """
    Prediksi semua gambar (.jpg/.jpeg/.png) di folder test_dir.
    Hasil disimpan ke CSV di dalam folder yang sama.
    """
    # Scan semua file gambar
    valid_ext = ('.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG')
    all_files = [f for f in os.listdir(test_dir) if f.endswith(valid_ext)]

    if not all_files:
        print(f"[WARN] ❌ Tidak ada gambar di folder '{test_dir}'.")
        print(f"        Taruh file .jpg/.png ke folder tersebut lalu jalankan lagi.")
        return None

    print(f"[INFO] Ditemukan {len(all_files)} gambar di '{test_dir}'")
    all_files = sorted(all_files)
    test_paths = [os.path.join(test_dir, f) for f in all_files]

    # Build tf.data pipeline
    def load_img(path):
        raw  = tf.io.read_file(path)
        img  = tf.image.decode_image(raw, channels=3, expand_animations=False)
        img  = tf.image.resize(img, img_size, method="bilinear")
        return tf.cast(img, tf.float32)

    test_ds = (
        tf.data.Dataset.from_tensor_slices(test_paths)
        .map(load_img, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    # Inference
    print("[INFO] Menjalankan prediksi...")
    all_probs   = np.concatenate([model.predict(batch, verbose=0) for batch in test_ds], axis=0)
    pred_idx    = np.argmax(all_probs, axis=1)
    pred_labels = [class_names[i] for i in pred_idx]
    confidences = [float(all_probs[i, pred_idx[i]]) for i in range(len(pred_idx))]

    # Simpan CSV
    result_df = pd.DataFrame({
        "filename"        : all_files,
        "predicted_label" : pred_labels,
        "confidence"      : [round(c, 4) for c in confidences]
    })

    out_csv = os.path.join(test_dir, "prediction_results.csv")
    result_df.to_csv(out_csv, index=False)
    print(f"[INFO] ✅ Hasil disimpan di: {out_csv}")

    # Visualisasi distribusi
    pred_dist = pd.Series(pred_labels).value_counts().sort_index()
    plt.figure(figsize=(10, 4))
    pred_dist.plot(kind='bar', color='#3498db', edgecolor='white', width=0.6)
    plt.title('Distribusi Prediksi', fontsize=13, fontweight='bold')
    plt.xlabel('Kelas Penyakit'); plt.ylabel('Jumlah Gambar')
    plt.xticks(rotation=30, ha='right')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

    print("\nHasil Prediksi:")
    print(result_df.to_string(index=False))
    return result_df


# Jalankan batch prediction
result_df = predict_all_in_folder(
    model       = inf_model,
    class_names = class_names_inf,
    test_dir    = CUSTOM_TEST_DIR
)

if result_df is not None:
    result_df.head(20)